# Full C3D object structure:

# Top Level (c)

## header

### points

- size
- frame_rate
- first_frame
- last_frame

### analogs

- size
- frame_rate
- first_frame
- last_frame

### events

- size
- events_time
- events_label

## parameters

### POINT

- __METADATA__
- USED
- SCALE
- RATE
- DATA_START
- FRAMES
- UNITS
- LABELS
- DESCRIPTIONS

### ANALOG

- __METADATA__
- USED
- LABELS
- GEN_SCALE
- SCALE
- OFFSET
- GAIN
- UNITS
- RATE
- RATIO
- FORMAT
- DESCRIPTIONS
- BITS

### FORCE_PLATFORM

- __METADATA__
- USED
- TYPE
- ZERO
- CORNERS
- ORIGIN
- CHANNEL
- ZEROS
- CAL_MATRIX

### MANUFACTURER

- __METADATA__
- COMPANY
- SOFTWARE
- VERSION_LABEL

### SUBJECTS

- __METADATA__
- USED
- LABEL_PREFIXES
- NAMES

### FORCE_STRUCTURE

- __METADATA__
- USED

### EVENT

- __METADATA__
- USED

## data

### points

numpy array: 4 x number of markers x number of point frames

### meta_points

- residuals
- camera_masks

### analogs

numpy array: 1 x 6*number of force plates x number of analog frames

In [2]:
import ezc3d
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
import os
from tqdm import tqdm

- type : OH [overhead], TQ [three quarter], SD [side], UD [under]
- strike : S [strike], B [ball]
- 최종 파일 저장 형태 : 이름_트라이이얼_오른왼쪽_몸무게_키_타입_볼스피드_스트라이크볼

In [3]:
day = '20240103'

In [4]:
kinematic_list = [i.replace('\\','/') for i in sorted(glob(f"c3d file/{day}/kinematic/**/*c3d"))]
kinetic_list = [i.replace('\\','/') for i in sorted(glob(f"c3d file/{day}/kinetic/**/*c3d"))]

kinematic_name = [os.path.basename(i).split('.')[0] for i in kinematic_list]
kinetic_name = [os.path.basename(i).split('.')[0] for i in kinetic_list]

In [5]:
players = pd.read_excel(f'c3d file/{day}/player_information.xlsx',index_col = 'trial')
players

,br_frame,ball_speed,strike,weight,height,type
trial,,,,,,
kimjiwoo_0001,554,116,S,85,183,OH
kimjiwoo_0002,610,117,S,85,183,OH
kimjiwoo_0003,628,117,S,85,183,OH
kimjiwoo_0004,559,126,S,85,183,OH
kimjiwoo_0005,593,117,S,85,183,OH
kimjiwoo_0006,496,117,S,85,183,OH
kimjiwoo_0007,785,117,S,85,183,OH
kimjiwoo_0008,713,116,S,85,183,OH
kimjiwoo_0009,524,116,S,85,183,OH


In [6]:
kinematic_c3d = {}
for i, name in zip(kinematic_list, kinematic_name):
    if 'right' in i:
        side = 'R'
    elif 'left' in i:
        side = 'L'
    
    weight = str(players.loc[name]['weight'])
    height = str(players.loc[name]['height'])
    pitchtype = players.loc[name]['type']
    ballspeed = str(players.loc[name]['ball_speed'])
    strikeball = players.loc[name]['strike']
    
    kinematic_c3d[f"{name}_{side}_{str(weight)}_{str(height)}_{pitchtype}_{ballspeed}_{strikeball}"] = ezc3d.c3d(i,extract_forceplat_data=True)
    
kinetic_c3d = {}
for i, name in zip(kinetic_list, kinetic_name):
    if 'right' in i:
        side = 'R'
    elif 'left' in i:
        side = 'L'
        
    weight = str(players.loc[name]['weight'])
    height = str(players.loc[name]['height'])
    pitchtype = players.loc[name]['type']
    ballspeed = str(players.loc[name]['ball_speed'])
    strikeball = players.loc[name]['strike']
    kinetic_c3d[f"{name}_{side}_{str(weight)}_{str(height)}_{pitchtype}_{ballspeed}_{strikeball}"] = ezc3d.c3d(i, extract_forceplat_data=True)

### Combine Kinematic C3D File & Force Plate C3D File

In [7]:
wrong = []
for file in tqdm(kinematic_c3d):
    try:
        theia = kinematic_c3d[file]
        device = kinetic_c3d[file]

        theia['header']['analogs']['size'] = device['header']['analogs']['size']
        theia['header']['analogs']['frame_rate'] = device['header']['analogs']['frame_rate']
        theia['header']['analogs']['first_frame'] = device['header']['analogs']['first_frame']
        theia['header']['analogs']['last_frame'] = device['header']['analogs']['last_frame']
        theia['parameters']['FORCE_PLATFORM'] = device['parameters']['FORCE_PLATFORM']
        theia['parameters']['PROCESSING'] = device['parameters']['PROCESSING']
        theia['parameters']['POINT'] = device['parameters']['POINT']
        theia['parameters']['POINT']['UNITS']['value'] = ['mm']
        theia['parameters']["FORCE_PLATFORM"]['CORNERS']['value'] = device['parameters']["FORCE_PLATFORM"]['CORNERS']['value']
        theia["parameters"]["ANALOG"] = device['parameters']['ANALOG']
        theia.add_parameter("ANALOG", "RATIO",description="Analog to Point Ratio", value=6)

        t_frame = theia['data']['points'].shape[-1] # 떼이아 프레임
        d_frame = device['data']['points'].shape[-1] # 아날로크 프레임

        if t_frame == d_frame:
            theia['data']['points'] = device['data']['points']
            theia['data']['analogs'] = device['data']['analogs']
            theia['data']['platform'] = device['data']['platform']
            
        elif t_frame < d_frame:
            theia['data']['points'] = device['data']['points'][:,:,:t_frame]
            theia['data']['analogs'] = device['data']['analogs'][:,:,:int(t_frame * 6)]
            theia['data']['platform'] = device['data']['platform']
            
        elif t_frame > d_frame:
            theia['data']['points'] = device['data']['points'][:,:,:d_frame]
            theia['data']['analogs'] = device['data']['analogs'][:,:,:int(d_frame * 6)]
            theia['data']['rotations'] = theia['data']['rotations'][:,:,:,:d_frame]
            theia['data']['platform'] = device['data']['platform']
            
        theia.write(f'c3d file/{day}/inte/{file}')
        
    except Exception:
        print('오류 :',file)
        wrong.append(file)

 68%|██████▊   | 13/19 [00:00<00:00, 117.74it/s]

오류 : kimjiwoo_0001_R_85_183_OH_116_S
오류 : kimjiwoo_0002_R_85_183_OH_117_S
오류 : kimjiwoo_0003_R_85_183_OH_117_S
오류 : kimjiwoo_0004_R_85_183_OH_126_S
오류 : kimjiwoo_0005_R_85_183_OH_117_S
오류 : kimjiwoo_0006_R_85_183_OH_117_S
오류 : kimjiwoo_0007_R_85_183_OH_117_S
오류 : kimjiwoo_0008_R_85_183_OH_116_S
오류 : kimjiwoo_0009_R_85_183_OH_116_S
오류 : kimjiwoo_0010_R_85_183_OH_117_S


100%|██████████| 19/19 [00:00<00:00, 56.56it/s] 


In [8]:
theia['parameters'].keys()

dict_keys(['MANUFACTURER', 'THEIA3D', 'PROCESSING', 'TRIAL', 'POINT', 'ROTATION', 'ANALOG', 'EVENT', 'DATA_BLOCKS', 'FORCE_PLATFORM'])

In [9]:
label = theia['parameters']['ROTATION']['LABELS']['value']
rot_mat = theia['data']['rotations']

In [10]:
ROT_MAT = {}
for i in range(rot_mat.shape[2]):
    ROT_MAT[label[i]] = rot_mat[:,:,i,:]

In [11]:
ROT_MAT['head_4X4'].shape

(4, 4, 1344)

In [12]:
wrong

['kimjiwoo_0001_R_85_183_OH_116_S',
 'kimjiwoo_0002_R_85_183_OH_117_S',
 'kimjiwoo_0003_R_85_183_OH_117_S',
 'kimjiwoo_0004_R_85_183_OH_126_S',
 'kimjiwoo_0005_R_85_183_OH_117_S',
 'kimjiwoo_0006_R_85_183_OH_117_S',
 'kimjiwoo_0007_R_85_183_OH_117_S',
 'kimjiwoo_0008_R_85_183_OH_116_S',
 'kimjiwoo_0009_R_85_183_OH_116_S',
 'kimjiwoo_0010_R_85_183_OH_117_S']

In [ ]:
# file = 'kjy_0009_R_86_184_OH_143_S'

# theia = kinematic_c3d[file]
# device = kinetic_c3d[file]


# theia['header']['analogs']['size'] = device['header']['analogs']['size']
# theia['header']['analogs']['frame_rate'] = device['header']['analogs']['frame_rate']
# theia['header']['analogs']['first_frame'] = device['header']['analogs']['first_frame']
# theia['header']['analogs']['last_frame'] = device['header']['analogs']['last_frame']
# theia['parameters']['FORCE_PLATFORM'] = device['parameters']['FORCE_PLATFORM']
# theia['parameters']['POINT'] = device['parameters']['POINT']
# theia['parameters']['POINT']['UNITS']['value'] = ['mm']
# theia['parameters']['PROCESSING'] = device['parameters']['PROCESSING']
# theia['parameters']["FORCE_PLATFORM"]['CORNERS']['value'] = device['parameters']["FORCE_PLATFORM"]['CORNERS']['value']
# theia["parameters"]["ANALOG"] = device['parameters']['ANALOG']
# theia.add_parameter("ANALOG", "RATIO",description="Analog to Point Ratio", value=6)

# t_frame = theia['data']['points'].shape[-1] # 떼이아 프레임
# d_frame = device['data']['points'].shape[-1] # 아날로크 프레임

# if t_frame == d_frame:
#     print('1', file)
#     theia['data']['points'] = device['data']['points']
#     theia['data']['analogs'] = device['data']['analogs']
#     theia['data']['platform'] = device['data']['platform']
    
# elif t_frame < d_frame:
#     print('2', file)
#     theia['data']['points'] = device['data']['points'][:,:,:t_frame]
#     theia['data']['analogs'] = device['data']['analogs'][:,:,:int(t_frame * 6)]
#     theia['data']['platform'] = device['data']['platform']
    
# elif t_frame > d_frame:
#     print('3', file)
#     theia['parameters']['TRIAL']['ACTUAL_END_FIELD']['value'] = np.array(([[d_frame],
#                                                                     [  0]]))
#     theia.add_parameter("ROTATION", "RATIO", 1)
#     theia['data']['points'] = device['data']['points'][:,:,:d_frame]
#     theia['data']['analogs'] = device['data']['analogs'][:,:,:int(d_frame * 6)]
#     theia['data']['rotations'] = theia['data']['rotations'][:,:,:,:d_frame]
#     theia['data']['platform'] = device['data']['platform']
    
# theia.write(f'c3d file/{day}/inte/{file}')

1 kjy_0009_R_86_184_OH_143_S



| c3d_f = c3d(file_list[0], extract_forceplat_data=True)             |
|--------------------------------------------------------------------|
|--------------------------------------------------------------------|
pf_0 = c3d_f["data"]["platform"][0]  # Select the first platform
pf_1 = c3d_f['data']['platform'][1]  # Select the second platform
pf_2 = c3d_f['data']['platform'][2]  # Select the third platform
|--------------------------------------------------------------------|
pf_0['unit_force']          # Units of forces
pf_0['unit_moment']         # Units of moments
pf_0['unit_position']       # Units of center of pressure
|--------------------------------------------------------------------|
pf_0['cal_matrix']          # Calibration matrix
pf_0['corners']             # Position of the corners
pf_0['origin']              # Position of the origin
|--------------------------------------------------------------------|
pf_0['force']               # Force data
pf_0['moment']              # Moment data
pf_0['center_of_pressure']  # Center of pressure data
pf_0['Tz']                  # Moment at center of pressure data